# Получение видео

### Импортирование библиотек

In [104]:
from googleapiclient.discovery import build
from google_auth_oauthlib.flow import InstalledAppFlow
from google.auth.transport.requests import Request

import os
import pickle
import string
import random
import json

### Функция инциализации сессии API

In [105]:
SCOPES = ["https://www.googleapis.com/auth/youtube.force-ssl"]

def youtube_authenticate():
    os.environ["OAUTHLIB_INSECURE_TRANSPORT"] = "1"
    api_service_name = "youtube"
    api_version = "v3"
    client_secrets_file = "credentials.json"
    creds = None
    
    if os.path.exists("token.pickle"):
        with open("token.pickle", "rb") as token:
            creds = pickle.load(token)

    if not creds or not creds.valid:
        if creds and creds.expired and creds.refresh_token:
            creds.refresh(Request())
        else:
            flow = InstalledAppFlow.from_client_secrets_file(client_secrets_file, SCOPES)
            creds = flow.run_local_server(port=0)
        with open("token.pickle", "wb") as token:
            pickle.dump(creds, token)

    return build(api_service_name, api_version, credentials=creds)

### Функция получения данных видео

In [106]:
def get_video_details(youtube, **kwargs):
    return youtube.videos().list(
        part="snippet,contentDetails,statistics",
        **kwargs
    ).execute()

### Запрос для поиска в API Google

In [108]:
def get_search_response(youtube, query: str, videos_num: int):
    search_response = youtube.search().list(
        q=query,
        part='snippet',
        maxResults=videos_num
    ).execute().get('items', [])

    return search_response

### Функция получения случайного запроса

In [ ]:
def get_random_query(length):
    return ''.join(random.choice(string.ascii_uppercase + string.digits) for _ in range(length))

### Получение только ID видео из запроса
Запрос выдаёт всё подряд, от видео до плейлистов. Эта функция фильтрует результат и возвращает ID только видео

In [109]:
def get_youtube_video_ids(search_response):
    return [
        search_result['id']['videoId']
        for search_result in search_response
        if search_result['id']['kind'] == 'youtube#video'
    ]

## Получение видео

In [110]:
youtube = youtube_authenticate()
search_response = get_search_response(youtube, get_random_query(length=3), 10)
video_ids = get_youtube_video_ids(search_response)

for id in video_ids:
    video_details = get_video_details(youtube, id=id)
    print(json.dumps(video_details, sort_keys=True, indent=4))

{
    "etag": "1X_qHV6N0Yg6BK4kNklT1EtDIhY",
    "items": [
        {
            "contentDetails": {
                "caption": "false",
                "contentRating": {},
                "definition": "hd",
                "dimension": "2d",
                "duration": "PT54M23S",
                "licensedContent": false,
                "projection": "rectangular"
            },
            "etag": "sJkYigW3A5-s_dw53QKfYDfK6nY",
            "id": "aocb9MXdNfY",
            "kind": "youtube#video",
            "snippet": {
                "categoryId": "10",
                "channelId": "UCmfF7JZv26UUKyRedViGIlw",
                "channelTitle": "H\u00d6R BERLIN",
                "description": "",
                "liveBroadcastContent": "none",
                "localized": {
                    "description": "",
                    "title": "Curtis Electronix - 3KZ / June 8 / 7pm-8pm"
                },
                "publishedAt": "2021-06-08T18:02:57Z",
                "thumb